# Load Agent

Load Agent (extracts the typical devices’ loads)
Predict the device load!


We do not include an explanation for the Load Agent since we do not consider the extracted typical load profile of every shiftable device as informative to the users

## **1. Load And Preprocess Data**

This part's only purpose is to load the data used in the Load Agent. This process is described in detail in the Preparation Agent. 

**Note: When computing the script with another Household than Household 1 you might need to adapt some parameters**

### **1.1 Initialize And Load Python Scripts**

In [3]:
import pandas as pd
import numpy as np
import os
import sqlite3
dir = 'D:/Master BWL HU/3. Semester/Seminar Information Systems/Seminar-Information-Systems-main'
os.chdir(dir)

from helper_functions import Helper
from agents import Preparation_Agent
import pandas as pd

helper = Helper()

dbfile  = "D:/Master BWL HU/3. Semester/Seminar Information Systems/Seminar-Information-Systems-main/home-assistant_v2.db"


C:\Users\chris\anaconda3\envs\recomm\lib\site-packages\xgboost\compat.py:36: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  from pandas import MultiIndex, Int64Index


### **1.2 Set Parameters For Pre-processing Step**

In [4]:
shiftable_devices = ["sensor.shellyplug_s_4022d88961b4_power", "sensor.shellyplug_s_4022d88984b8_power"]
date = '2023-03-15'
activity_devices = ["light.olaf_der_gesundigte", "light.blub", "light.extended_color_light_1", "switch.shellyplug_s_80646f824e22", 
                    "light.bogenleuchte", "light.hue_color_candle_1", "light.hue_color_candle_2"]


truncation_params = {
    'features': 'all', 
    'factor': 1.5, 
    'verbose': 1
}

scale_params = {
    'features': 'all', 
    'kind': 'MinMax', 
    'verbose': 1
}

aggregate_params = {
    'resample_param': '60T'
}
#update with active appliances attributes_ids
activity_params = {
    'active_appliances': shiftable_devices,
    'threshold': .15
}

time_params = {
    'features': ['hour', 'day_name']
}

activity_lag_params = {
    'features': ['activity'],
    'lags': [24, 48, 72]
}

device_params = {
    'threshold': 1
}

load_pipe_params = {
    'truncate': truncation_params,
    'scale': scale_params,
    'aggregate': aggregate_params,
    'shiftable_devices': shiftable_devices, 
    'device': device_params
}

### **1.3 Pre-process Data For Input In Device_Usage Agent**

In [5]:
# calling the preparation pipelin
import pandas as pd
prep = Preparation_Agent(dbfile, shiftable_devices, activity_devices, number_days)
output, scaled, df = prep.pipeline_load(prep.input, load_pipe_params)
output

,sensor.shellyplug_s_4022d88961b4_power,sensor.shellyplug_s_4022d88984b8_power
last_updated_ts,,
2023-01-21 11:00:00,1054.230719,0.0
2023-01-21 12:00:00,575.244490,0.0
2023-01-21 13:00:00,0.000000,0.0
2023-01-21 14:00:00,0.000000,0.0
2023-01-21 15:00:00,0.000000,0.0
...,...,...
2023-03-23 19:00:00,0.000000,0.0
2023-03-23 20:00:00,0.000000,0.0
2023-03-23 21:00:00,0.000000,0.0


In [6]:
df = output
df

,sensor.shellyplug_s_4022d88961b4_power,sensor.shellyplug_s_4022d88984b8_power
last_updated_ts,,
2023-01-21 11:00:00,1054.230719,0.0
2023-01-21 12:00:00,575.244490,0.0
2023-01-21 13:00:00,0.000000,0.0
2023-01-21 14:00:00,0.000000,0.0
2023-01-21 15:00:00,0.000000,0.0
...,...,...
2023-03-23 19:00:00,0.000000,0.0
2023-03-23 20:00:00,0.000000,0.0
2023-03-23 21:00:00,0.000000,0.0


## **2.  Constructing the Load Agent**

### **2.1 Initialize Agent**

First we define the **Load Agent class**. It takes as input the data generated by the prep.pipeline_usage function computed above.

In [7]:
class Load_Agent:
    def __init__(self, load_input_df):
        self.input = load_input_df

### Truncate start and end dates if observations are missing
Truncating days where there are less than 24 reported hours

In [8]:
# selecting the correct data, identifying device runs, creating load profiles
# -------------------------------------------------------------------------------------------
def prove_start_end_date(self, df, date):
    import pandas as pd

    start_date = (df.index[0]).strftime("%Y-%m-%d")
    end_date = date
    
    if len(df.loc[start_date]) < 24:
        start_date = (pd.to_datetime(start_date) + pd.Timedelta(days=1)).strftime(
            "%Y-%m-%d"
        )
        df = df[start_date:end_date]
    else:
        df = df[:end_date]

    if end_date not in df.index:
        return df
    
    if len(df.loc[end_date]) < 24:
        end_new = (pd.to_datetime(end_date) - pd.Timedelta(days=1)).strftime(
            "%Y-%m-%d"
        )
        df = df[:end_new]
    else:
        df = df[:end_date]
    return df

# add to Load agent
setattr(Load_Agent, 'prove_start_end_date', prove_start_end_date)
del prove_start_end_date 

In [9]:
Load_Agent_i = Load_Agent(df) 
df = Load_Agent_i.prove_start_end_date(df, date)

In [10]:
df

,sensor.shellyplug_s_4022d88961b4_power,sensor.shellyplug_s_4022d88984b8_power
last_updated_ts,,
2023-01-22 00:00:00,0.0,0.0
2023-01-22 01:00:00,0.0,0.0
2023-01-22 02:00:00,0.0,0.0
2023-01-22 03:00:00,0.0,0.0
2023-01-22 04:00:00,0.0,0.0
...,...,...
2023-03-15 19:00:00,0.0,0.0
2023-03-15 20:00:00,0.0,0.0
2023-03-15 21:00:00,0.0,0.0


### Exlude today, use up until yesterday

Only use dates excluding the date we want to predict, so use all up until "yesterday"

In [11]:
def df_yesterday_date(self, df, date):
    import pandas as pd

    yesterday = (pd.to_datetime(date) - pd.Timedelta(days=1)).strftime("%Y-%m-%d")
    return df[:yesterday]
# add to Load agent
setattr(Load_Agent, 'df_yesterday_date', df_yesterday_date)
del df_yesterday_date 

In [12]:
Load_Agent_i = Load_Agent(df) 
df = Load_Agent_i.df_yesterday_date(df, date)
df

,sensor.shellyplug_s_4022d88961b4_power,sensor.shellyplug_s_4022d88984b8_power
last_updated_ts,,
2023-01-22 00:00:00,0.0,0.0
2023-01-22 01:00:00,0.0,0.0
2023-01-22 02:00:00,0.0,0.0
2023-01-22 03:00:00,0.0,0.0
2023-01-22 04:00:00,0.0,0.0
...,...,...
2023-03-14 19:00:00,0.0,0.0
2023-03-14 20:00:00,0.0,0.0
2023-03-14 21:00:00,0.0,0.0


In [13]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):  # more options can be specified also
    print(df['sensor.shellyplug_s_4022d88984b8_power'])

last_updated_ts
2023-01-22 00:00:00       0.000000
2023-01-22 01:00:00       0.000000
2023-01-22 02:00:00       0.000000
2023-01-22 03:00:00       0.000000
2023-01-22 04:00:00       0.000000
2023-01-22 05:00:00       0.000000
2023-01-22 06:00:00       0.000000
2023-01-22 07:00:00       0.000000
2023-01-22 08:00:00       0.000000
2023-01-22 09:00:00       0.000000
2023-01-22 10:00:00     787.121076
2023-01-22 11:00:00      78.920563
2023-01-22 12:00:00      77.205085
2023-01-22 13:00:00       0.000000
2023-01-22 14:00:00       0.000000
2023-01-22 15:00:00       0.000000
2023-01-22 16:00:00       0.000000
2023-01-22 17:00:00       0.000000
2023-01-22 18:00:00       0.000000
2023-01-22 19:00:00       0.000000
2023-01-22 20:00:00       0.000000
2023-01-22 21:00:00       0.000000
2023-01-22 22:00:00       0.000000
2023-01-22 23:00:00       0.000000
2023-01-23 00:00:00       0.000000
2023-01-23 01:00:00       0.000000
2023-01-23 02:00:00       0.000000
2023-01-23 03:00:00       0.000000
2023

### Creating df_hours
Creating raw load profile for each hour of each day
Creating new data frame df_hours

In [14]:
def load_profile_raw(self, df, shiftable_devices):
    import pandas as pd

    hours = []
    for hour in range(1, 25):
        hours.append("h" + str(hour))
    df_hours = {}

    for idx, appliance in enumerate(
        shiftable_devices
    ):
        df_hours[appliance] = pd.DataFrame(index=None, columns=hours)
        column = df[appliance]

        for i in range(len(column)):

            if (i == 0) and (column[0] > 0):
                df_hours[appliance].loc[0, "h" + str(1)] = column[0]

            if (column[i - 1] == 0) and (column[i] > 0):
                for j in range(0, 24):
                    if (i + j) < len(column):
                        if column[i + j] > 0:
                            df_hours[appliance].loc[i, "h" + str(j + 1)] = column[
                                i + j
                            ]
    return df_hours

# add to Load agent
setattr(Load_Agent, 'load_profile_raw', load_profile_raw)
del load_profile_raw 

In [15]:
Load_Agent_i = Load_Agent(df) 
df_hours = Load_Agent_i.load_profile_raw(df, shiftable_devices)

In [16]:
df_hours['sensor.shellyplug_s_4022d88984b8_power']

,h1,h2,h3,h4,h5,h6,h7,h8,h9,h10,...,h15,h16,h17,h18,h19,h20,h21,h22,h23,h24
10,787.121076,78.920563,77.205085,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
88,523.524417,78.554765,106.418,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,312.911,86.544132
110,312.911,86.544132,5.738462,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
179,1013.791065,261.008879,114.481468,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
203,1096.723361,100.803599,92.078905,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
248,22.739437,52.700883,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
393,989.187761,129.944251,84.438995,180.259459,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
441,280.729276,354.291811,103.325673,120.534862,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
535,1648.613568,140.347923,89.131158,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
545,160.185431,73.41306,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [17]:
df_hours['sensor.shellyplug_s_4022d88961b4_power']

,h1,h2,h3,h4,h5,h6,h7,h8,h9,h10,...,h15,h16,h17,h18,h19,h20,h21,h22,h23,h24
14,10.391667,904.461303,291.047692,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
58,513.181818,964.683077,457.847761,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
104,1401.767143,740.812648,1.146857,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
129,1095.196296,722.7816,484.856872,215.556118,514.593627,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,931.18655
152,931.18655,574.841667,NaN,NaN,NaN,683.413855,673.6,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
157,683.413855,673.6,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,828.442745,249.649485,NaN
178,828.442745,249.649485,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
224,675.972487,634.100769,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
296,943.251295,568.832735,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
344,984.72067,524.311638,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


### Cleaning df_hours
Cleaning the new data frame df_hours

In [18]:
def load_profile_cleaned(self, df_hours):
    import numpy as np

    for app in df_hours.keys():
        for i in df_hours[app].index:
            for j in df_hours[app].columns:
                if np.isnan(df_hours[app].loc[i, j]):
                    df_hours[app].loc[i, j:] = 0
    return df_hours

# add to Load agent
setattr(Load_Agent, 'load_profile_cleaned', load_profile_cleaned)
del load_profile_cleaned 

In [19]:
Load_Agent_i = Load_Agent(df) 
df_hours = Load_Agent_i.load_profile_cleaned(df_hours)

In [20]:
df_hours

{'sensor.shellyplug_s_4022d88961b4_power':                h1          h2          h3          h4          h5 h6 h7 h8 h9  \
 14      10.391667  904.461303  291.047692           0           0  0  0  0  0   
 58     513.181818  964.683077  457.847761           0           0  0  0  0  0   
 104   1401.767143  740.812648    1.146857           0           0  0  0  0  0   
 129   1095.196296    722.7816  484.856872  215.556118  514.593627  0  0  0  0   
 152     931.18655  574.841667           0           0           0  0  0  0  0   
 157    683.413855       673.6           0           0           0  0  0  0  0   
 178    828.442745  249.649485           0           0           0  0  0  0  0   
 224    675.972487  634.100769           0           0           0  0  0  0  0   
 296    943.251295  568.832735           0           0           0  0  0  0  0   
 344     984.72067  524.311638           0           0           0  0  0  0  0   
 392     933.18902  234.519104           0           0  

In [21]:
df_hours['sensor.shellyplug_s_4022d88984b8_power']

,h1,h2,h3,h4,h5,h6,h7,h8,h9,h10,...,h15,h16,h17,h18,h19,h20,h21,h22,h23,h24
10,787.121076,78.920563,77.205085,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
88,523.524417,78.554765,106.418,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
110,312.911,86.544132,5.738462,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
179,1013.791065,261.008879,114.481468,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
203,1096.723361,100.803599,92.078905,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
248,22.739437,52.700883,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
393,989.187761,129.944251,84.438995,180.259459,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
441,280.729276,354.291811,103.325673,120.534862,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
535,1648.613568,140.347923,89.131158,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
545,160.185431,73.41306,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Load Profiles


In [22]:
def load_profile(self, df_hours, shiftable_devices):
    import pandas as pd

    hours = df_hours[shiftable_devices[0]].columns
    loads = pd.DataFrame(columns=hours)

    for app in df_hours.keys():
        app_mean = df_hours[app].apply(lambda x: x.mean(), axis=0)
        for hour in app_mean.index:
            loads.loc[app, hour] = app_mean[hour]

    loads = loads.fillna(0)
    return loads
# add to Load agent
setattr(Load_Agent, 'load_profile', load_profile)
del load_profile 

In [23]:
Load_Agent_i = Load_Agent(df) 
loads = Load_Agent_i.load_profile(df_hours, shiftable_devices)
loads

,h1,h2,h3,h4,h5,h6,h7,h8,h9,h10,...,h15,h16,h17,h18,h19,h20,h21,h22,h23,h24
sensor.shellyplug_s_4022d88961b4_power,896.651812,628.536985,44.671280,7.185204,17.153121,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
sensor.shellyplug_s_4022d88984b8_power,667.202075,227.954989,73.118868,31.086815,20.473722,4.732377,4.351459,0.303472,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


## Evaluation Load

### Get true loads

In [24]:
# evaluating the performance of the load agent
# -------------------------------------------------------------------------------------------
def get_true_loads(self, shiftable_devices):
    true_loads = self.load_profile_raw(self.input, shiftable_devices)
    true_loads = self.load_profile_cleaned(true_loads)
    for device, loads in true_loads.items():
        true_loads[device].rename(
            index=dict(enumerate(self.input.index)), inplace=True
        )
    return true_loads
# add to Load agent
setattr(Load_Agent, 'get_true_loads', get_true_loads)
del get_true_loads 

In [25]:
Load_Agent_i = Load_Agent(df) 
true_loads = Load_Agent_i.get_true_loads(shiftable_devices)
true_loads

{'sensor.shellyplug_s_4022d88961b4_power':                               h1          h2          h3          h4  \
 2023-01-22 14:00:00    10.391667  904.461303  291.047692           0   
 2023-01-24 10:00:00   513.181818  964.683077  457.847761           0   
 2023-01-26 08:00:00  1401.767143  740.812648    1.146857           0   
 2023-01-27 09:00:00  1095.196296    722.7816  484.856872  215.556118   
 2023-01-28 08:00:00    931.18655  574.841667           0           0   
 2023-01-28 13:00:00   683.413855       673.6           0           0   
 2023-01-29 10:00:00   828.442745  249.649485           0           0   
 2023-01-31 08:00:00   675.972487  634.100769           0           0   
 2023-02-03 08:00:00   943.251295  568.832735           0           0   
 2023-02-05 08:00:00    984.72067  524.311638           0           0   
 2023-02-07 08:00:00    933.18902  234.519104           0           0   
 2023-02-07 15:00:00   577.530159           0           0           0   
 2023-02-

In [26]:
true_loads['sensor.shellyplug_s_4022d88984b8_power']

,h1,h2,h3,h4,h5,h6,h7,h8,h9,h10,...,h15,h16,h17,h18,h19,h20,h21,h22,h23,h24
2023-01-22 10:00:00,787.121076,78.920563,77.205085,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-01-25 16:00:00,523.524417,78.554765,106.418,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-01-26 14:00:00,312.911,86.544132,5.738462,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-01-29 11:00:00,1013.791065,261.008879,114.481468,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-01-30 11:00:00,1096.723361,100.803599,92.078905,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-02-01 08:00:00,22.739437,52.700883,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-02-07 09:00:00,989.187761,129.944251,84.438995,180.259459,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-02-09 09:00:00,280.729276,354.291811,103.325673,120.534862,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-02-13 07:00:00,1648.613568,140.347923,89.131158,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-02-13 17:00:00,160.185431,73.41306,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Create pipline

In [27]:
# pipeline function: creating typical load profiles
# -------------------------------------------------------------------------------------------
def pipeline(self, df, date, shiftable_devices):
    df = self.prove_start_end_date(df, date)
    df = self.df_yesterday_date(df, date)
    df_hours = self.load_profile_raw(df, shiftable_devices)
    df_hours = self.load_profile_cleaned(df_hours)
    loads = self.load_profile(df_hours, shiftable_devices)
    return loads
# add to Load agent
setattr(Load_Agent, 'pipeline', pipeline)
del pipeline 


In [28]:
df = output

In [29]:
Load_Agent_i = Load_Agent(df) 
loads = Load_Agent_i.pipeline(df, date, shiftable_devices)
loads

,h1,h2,h3,h4,h5,h6,h7,h8,h9,h10,...,h15,h16,h17,h18,h19,h20,h21,h22,h23,h24
sensor.shellyplug_s_4022d88961b4_power,896.651812,628.536985,44.671280,7.185204,17.153121,0.000000,0.000000,0.000000,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
sensor.shellyplug_s_4022d88984b8_power,667.202075,227.954989,73.118868,31.086815,20.473722,4.732377,4.351459,0.303472,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


### Evaluation

In [30]:
true_loads = Load_Agent_i.get_true_loads(shiftable_devices)
device = 'sensor.shellyplug_s_4022d88984b8_power'
true_loads

{'sensor.shellyplug_s_4022d88961b4_power':                               h1          h2          h3          h4  \
 2023-01-21 11:00:00  1054.230719   575.24449           0           0   
 2023-01-22 14:00:00    10.391667  904.461303  291.047692           0   
 2023-01-24 10:00:00   513.181818  964.683077  457.847761           0   
 2023-01-26 08:00:00  1401.767143  740.812648    1.146857           0   
 2023-01-27 09:00:00  1095.196296    722.7816  484.856872  215.556118   
 2023-01-28 08:00:00    931.18655  574.841667           0           0   
 2023-01-28 13:00:00   683.413855       673.6           0           0   
 2023-01-29 10:00:00   828.442745  249.649485           0           0   
 2023-01-31 08:00:00   675.972487  634.100769           0           0   
 2023-02-03 08:00:00   943.251295  568.832735           0           0   
 2023-02-05 08:00:00    984.72067  524.311638           0           0   
 2023-02-07 08:00:00    933.18902  234.519104           0           0   
 2023-02-

In [31]:
true_loads[device] = Load_Agent_i.prove_start_end_date(true_loads[device], date)

In [32]:
true_loads[device]

,h1,h2,h3,h4,h5,h6,h7,h8,h9,h10,...,h15,h16,h17,h18,h19,h20,h21,h22,h23,h24
2023-01-25 16:00:00,523.524417,78.554765,106.418,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-01-26 14:00:00,312.911,86.544132,5.738462,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-01-29 11:00:00,1013.791065,261.008879,114.481468,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-01-30 11:00:00,1096.723361,100.803599,92.078905,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-02-01 08:00:00,22.739437,52.700883,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-02-07 09:00:00,989.187761,129.944251,84.438995,180.259459,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-02-09 09:00:00,280.729276,354.291811,103.325673,120.534862,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-02-13 07:00:00,1648.613568,140.347923,89.131158,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-02-13 17:00:00,160.185431,73.41306,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2023-02-15 08:00:00,1222.68209,761.163123,98.163144,96.535506,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [34]:
def evaluate(self, shiftable_devices, date = '2023-01-01', metric="mse", aggregate=True, evaluation=False):
    from tqdm import tqdm
    import pandas as pd
    import numpy as np
    tqdm.pandas()

    if metric == "mse":
        import sklearn.metrics

        metric = sklearn.metrics.mean_squared_error

    true_loads = self.get_true_loads(shiftable_devices)

    scores = {}
    if not evaluation:
        for device in shiftable_devices:
            true_loads[device] = self.prove_start_end_date(true_loads[device], date)
        scores[device] = true_loads[device].progress_apply(
            lambda row: metric(
                row.values,
                self.pipeline(
                    self.input, str(row.name)[:10], [device]
                ).values.reshape(
                    -1,
                ),
            ),
                axis=1,
            )
    else:
        for device in shiftable_devices:
            scores[device] = {}
            for idx in tqdm(true_loads[device].index):
                date = str(idx)[:10]
                y_true = true_loads[device].loc[idx, :].values
                try:
                    y_hat = (
                        evaluation[date]
                        .loc[device]
                        .values.reshape(
                            -1,
                        )
                    )
                except KeyError:
                    try:
                        y_hat = self.pipeline(
                            self.input, date, [device]
                        ).values.reshape(
                            -1,
                        )
                    except:
                        y_hat = np.full(24, 0)
                scores[device][idx] = metric(y_true, y_hat)
            scores[device] = pd.Series(scores[device])

    if aggregate:
        scores = {device: scores_df.mean() for device, scores_df in scores.items()}
    return scores

# add to Load agent
setattr(Load_Agent, 'evaluate', evaluate)
del evaluate 

evaluation just runs if the data later on is available. So we run it on 01.01.2023, because for 02.01. we donot have data for each hour

In [35]:
df = output
df

,sensor.shellyplug_s_4022d88961b4_power,sensor.shellyplug_s_4022d88984b8_power
last_updated_ts,,
2023-01-21 11:00:00,1054.230719,0.0
2023-01-21 12:00:00,575.244490,0.0
2023-01-21 13:00:00,0.000000,0.0
2023-01-21 14:00:00,0.000000,0.0
2023-01-21 15:00:00,0.000000,0.0
...,...,...
2023-03-23 19:00:00,0.000000,0.0
2023-03-23 20:00:00,0.000000,0.0
2023-03-23 21:00:00,0.000000,0.0


In [36]:
Load_Agent_i = Load_Agent(df) 
scores = Load_Agent_i.evaluate(shiftable_devices, date, evaluation=False)
scores

100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [00:00<00:00, 33.82it/s]


{'sensor.shellyplug_s_4022d88984b8_power': 13885.820143484238}

# Complete Load Agent

In [2]:
###################################################################################################
# load agent ######################################################################################
###################################################################################################
class Load_Agent:
    def __init__(self, load_input_df):
        self.input = load_input_df

    # selecting the correct data, identifying device runs, creating load profiles
    # -------------------------------------------------------------------------------------------
    def prove_start_end_date(self, df, date):
        import pandas as pd

        start_date = (df.index[0]).strftime("%Y-%m-%d")
        end_date = date

        if len(df.loc[start_date]) < 24:
            start_date = (pd.to_datetime(start_date) + pd.Timedelta(days=1)).strftime(
                "%Y-%m-%d"
            )
            df = df[start_date:end_date]
        else:
            df = df[:end_date]
        
        if end_date not in df.index:
            return df

        if len(df.loc[end_date]) < 24:
            end_new = (pd.to_datetime(end_date) - pd.Timedelta(days=1)).strftime(
                "%Y-%m-%d"
            )
            df = df[:end_new]
        else:
            df = df[:end_date]
        return df

    def df_yesterday_date(self, df, date):
        import pandas as pd

        yesterday = (pd.to_datetime(date) - pd.Timedelta(days=1)).strftime("%Y-%m-%d")
        return df[:yesterday]

    def load_profile_raw(self, df, shiftable_devices):
        import pandas as pd

        hours = []
        for hour in range(1, 25):
            hours.append("h" + str(hour))
        df_hours = {}

        for idx, appliance in enumerate(
            shiftable_devices
        ):
            df_hours[appliance] = pd.DataFrame(index=None, columns=hours)
            column = df[appliance]

            for i in range(len(column)):

                if (i == 0) and (column[0] > 0):
                    df_hours[appliance].loc[0, "h" + str(1)] = column[0]

                elif (column[i - 1] == 0) and (column[i] > 0):
                    for j in range(0, 24):
                        if (i + j) < len(column):
                            if column[i + j] > 0:
                                df_hours[appliance].loc[i, "h" + str(j + 1)] = column[
                                    i + j
                                ]
        return df_hours

    def load_profile_cleaned(self, df_hours):
        import numpy as np

        for app in df_hours.keys():
            for i in df_hours[app].index:
                for j in df_hours[app].columns:
                    if np.isnan(df_hours[app].loc[i, j]):
                        df_hours[app].loc[i, j:] = 0
        return df_hours

    def load_profile(self, df_hours, shiftable_devices):
        import pandas as pd

        hours = df_hours[shiftable_devices[0]].columns
        loads = pd.DataFrame(columns=hours)

        for app in df_hours.keys():
            app_mean = df_hours[app].apply(lambda x: x.mean(), axis=0)
            for hour in app_mean.index:
                loads.loc[app, hour] = app_mean[hour]

        loads = loads.fillna(0)
        return loads

    # evaluating the performance of the load agent
    # -------------------------------------------------------------------------------------------
    def get_true_loads(self, shiftable_devices):
        true_loads = self.load_profile_raw(self.input, shiftable_devices)
        true_loads = self.load_profile_cleaned(true_loads)
        for device, loads in true_loads.items():
            true_loads[device].rename(
                index=dict(enumerate(self.input.index)), inplace=True
            )
        return true_loads

    def evaluate(self, shiftable_devices, date = '2023-01-01', metric="mse", aggregate=True, evaluation=False):
        from tqdm import tqdm
        import pandas as pd
        import numpy as np
        tqdm.pandas()

        if metric == "mse":
            import sklearn.metrics

            metric = sklearn.metrics.mean_squared_error

        true_loads = self.get_true_loads(shiftable_devices)

        scores = {}
        if not evaluation:
            for device in shiftable_devices:
                true_loads[device] = self.prove_start_end_date(true_loads[device], date)
            scores[device] = true_loads[device].progress_apply(
                lambda row: metric(
                    row.values,
                    self.pipeline(
                        self.input, str(row.name)[:10], [device]
                    ).values.reshape(
                        -1,
                    ),
                ),
                    axis=1,
                )
        else:
            for device in shiftable_devices:
                scores[device] = {}
                for idx in tqdm(true_loads[device].index):
                    date = str(idx)[:10]
                    y_true = true_loads[device].loc[idx, :].values
                    try:
                        y_hat = (
                            evaluation[date]
                            .loc[device]
                            .values.reshape(
                                -1,
                            )
                        )
                    except KeyError:
                        try:
                            y_hat = self.pipeline(
                                self.input, date, [device]
                            ).values.reshape(
                                -1,
                            )
                        except:
                            y_hat = np.full(24, 0)
                    scores[device][idx] = metric(y_true, y_hat)
                scores[device] = pd.Series(scores[device])

        if aggregate:
            scores = {device: scores_df.mean() for device, scores_df in scores.items()}
        return scores

    # pipeline function: creating typical load profiles
    # -------------------------------------------------------------------------------------------
    def pipeline(self, df, date, shiftable_devices):
        df = self.prove_start_end_date(df, date)
        df = self.df_yesterday_date(df, date)
        df_hours = self.load_profile_raw(df, shiftable_devices)
        df_hours = self.load_profile_cleaned(df_hours)
        loads = self.load_profile(df_hours, shiftable_devices)
        return loads

In [36]:
# calling the preparation pipelin
import pandas as pd
prep = Preparation_Agent(dbfile, shiftable_devices)
output, scaled, df = prep.pipeline_load(prep.input, load_pipe_params)
output
df = output

In [38]:
shiftable_devices = ["sensor.shellyplug_s_4022d88961b4_power", "sensor.shellyplug_s_4022d88984b8_power"]
load = Load_Agent(df)

date = '2023-01-09'

output = load.pipeline(df, date, shiftable_devices)
output

,h1,h2,h3,h4,h5,h6,h7,h8,h9,h10,...,h15,h16,h17,h18,h19,h20,h21,h22,h23,h24
sensor.shellyplug_s_4022d88961b4_power,628.452286,550.879960,104.48717,0.186528,0.000000,0.000000,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
sensor.shellyplug_s_4022d88984b8_power,663.272762,249.475287,88.61876,78.469345,230.205367,16.102368,0.859524,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
